In [926]:
import pandas as pd
import ast
import numpy as np

pd.options.display.float_format = '{:.1%}'.format
# pd.options.display.float_format = '{:.3f}'.format


In [894]:
def return_metrics(filename):
    df = pd.read_csv(filename, quotechar='"')

    rename_dict = {
        'accuracy': 'train_acc',
        'accuracy_test': 'test_acc'
    }

    df = df.rename(columns=rename_dict)

    df["Name"] = df.Name.apply(lambda x: x[:-2])

    df_metrics = df.groupby(['Name'], as_index=False).agg(
                        {'test_acc':['mean','std'], 'train_acc':['mean','std']})

    df_metrics.xs("test_acc", axis=1)
    df_metrics["mean_vs_std"] = df_metrics.xs("test_acc", axis=1)["std"]/df_metrics.xs("test_acc", axis=1)["mean"]
    df_metrics.loc["Mean"] = df_metrics.mean()

    return df_metrics

In [895]:
def flip_df(df):
    upper = df.T.to_numpy()
    lower = df.to_numpy()
    n = len(upper) + 1
    out = np.eye(n)*np.nan
    out[np.triu_indices(n, 1)] = upper[np.triu_indices(n - 1)]
    out[np.tril_indices(n, -1)] = lower[np.tril_indices(n-1)]

    return out

def metrics_genres(filename):
    df = pd.read_csv(filename, quotechar='"')

    rename_dict = {
        'accuracy': 'train_acc',
        'accuracy_test': 'test_acc'
    }

    df = df.rename(columns=rename_dict)

    df["Name"] = df.Name.apply(lambda x: x[:-2])

    # df_metrics = df.groupby(['Name'], as_index=False).agg(
    #                     {'test_acc':['mean','std'], 'train_acc':['mean','std']})
    df.sort_values('Name', axis=0)

    df = df.groupby(['Name', 'genres'], as_index=False).agg({'test_acc':['mean','std']})
    df.columns = pd.Index(['Name', 'genres', 'mean', 'std'])

    df = df.drop('Name', axis=1)
    # df.genres.apply(lambda x: ast.literal_eval(x)[0])
    df['genre1'] = df.genres.apply(lambda x: ast.literal_eval(x)[0])
    df['genre2'] = df.genres.apply(lambda x: ast.literal_eval(x)[1])
    df = df.drop('genres', axis=1)

    df_multiindex = df.loc[:, ['genre1', 'genre2']]
    # df.stack(2)

    df_multiindex = pd.MultiIndex.from_arrays(df_multiindex.to_numpy().T)
    df_multiindex

    df = df.set_index(df_multiindex).drop(['genre1', 'genre2'], axis=1)

    df = df.unstack().T

    df_std = df.xs('std', level=0)
    df = df.xs('mean', level=0)

    df_index = df.columns.append(pd.Index([df.index[-1]]))
    
    df = flip_df(df)
    df_std = flip_df(df_std)


    df = pd.DataFrame(df).set_axis(df_index, axis=1).set_axis(df_index, axis=0)

    df.loc['mean'] = df.mean(numeric_only=True)
    df.loc['std'] = df.std(numeric_only=True)

    return df

In [896]:
def return_metrics_cnn(filename):
    df = pd.read_csv(filename, quotechar='"')

    df["Name"] = df.Name.apply(lambda x: x[:-2])

    df_metrics = df.groupby(['Name'], as_index=False).agg(
                        {'accuracy_test':['mean','std'], 'accuracy':['mean','std']})

    df_metrics.xs("accuracy_test", axis=1)
    df_metrics["mean_vs_std"] = df_metrics.xs("accuracy_test", axis=1)["mean"]/df_metrics.xs("accuracy_test", axis=1)["std"]
    df_metrics.loc["Mean"] = df_metrics.mean()

    return df_metrics


In [897]:
def return_metrics_gs(filename):
    df = pd.read_csv(filename, quotechar='"')

    df["Name"] = df.Name.apply(lambda x: x[:-2])

    df_metrics = df.groupby(['optim', 'id'], as_index=False).agg(
                        {'test_acc':['mean','std'], 'train_acc':['mean','std'], 'loss':['mean','std']})

    df_metrics.xs("test_acc", axis=1)
    df_metrics["mean_vs_std"] = df_metrics.xs("test_acc", axis=1)["mean"]/df_metrics.xs("test_acc", axis=1)["std"]
    df_metrics["mean_vs_std_loss"] = df_metrics.xs("loss", axis=1)["mean"]/df_metrics.xs("loss", axis=1)["std"]
    df_metrics.loc["Mean"] = df_metrics.mean()

    return df_metrics

In [898]:
def metrics_comparison(csv_list, axis=[256, 512, 1024]):
    # axis = [256, 512, 1024]

    df_mean_list = []
    df_std_list = []

    for file in csv_list:
        df_new = pd.read_csv(file, quotechar='"')

        rename_dict = {
            'accuracy': 'train_acc',
            'accuracy_test': 'test_acc'
        }

        df_new = df_new.rename(columns=rename_dict)

        df_new["Name"] = df_new.Name.apply(lambda x: x[:-2])

        df_mean = df_new.groupby(['Name'], axis=0, as_index=False).mean(numeric_only=True)
        df_std = df_new.groupby(['Name'], axis=0, as_index=False).std(numeric_only=True)
        
        df_mean_list.append(df_mean)
        df_std_list.append(df_std)

    df_mean = pd.concat(df_mean_list, keys=axis)
    df_std = pd.concat(df_std_list, keys=axis)

    return df_mean, df_std

def merge_metrics(df):
    index1 = df.T.swaplevel(axis=1).columns.get_level_values(level=1).to_list()
    index2 = df.T.loc['Name'].to_numpy().tolist()
    mltindex = pd.MultiIndex.from_arrays([index1, index2])

    df = df.drop(columns=['Name'])

    df.set_index(mltindex, inplace=True)

    df = df.swaplevel(axis=0)
    df = df.reindex(sorted(df.index), axis=0)
    df = df.T

    return df

## Ficheros y csv

### QCNNs

In [899]:
f00 = "4782-0-0.csv"
f10 = "4849-1-0.csv"
f20 = "4863-2-0.csv"

f01 = "4783-0-1.csv"
f11 = "4847-1-1.csv"
f21 = "4864-2-1.csv"

f02 = "4829-0-2.csv"
f12 = "4850-1-2.csv"
f22 = "4865-2-2.csv"

muest_csv_0 = [f00, f10, f20]
char_csv_0 = [f00, f01, f02]

muest_csv_1 = [f01, f11, f21]
char_csv_1 = [f10, f11, f12]

muest_csv_2 = [f02, f12, f22]
char_csv_2 = [f20, f21, f22]

axis1 = [1, 10, 20]
axis0 = [256, 512, 1024]

### CNNs

In [900]:
cnnf00 = "5237-0-0.csv"
cnnf10 = "5250-1-0.csv"
cnnf20 = "5249-2-0.csv"

cnnf01 = "5246-0-1.csv"
cnnf11 = "5258-1-1.csv"
cnnf21 = "5260-2-1.csv"

cnnf02 = "5247-0-2.csv"
cnnf12 = "5259-1-2.csv"
cnnf22 = "5261-2-2.csv"

cnn_muest_csv_0 = [cnnf00, cnnf10, cnnf20]
cnn_char_csv_0 = [cnnf00, cnnf01, cnnf02]

cnn_muest_csv_1 = [cnnf01, cnnf11, cnnf21]
cnn_char_csv_1 = [cnnf10, cnnf11, cnnf12]

cnn_muest_csv_2 = [cnnf02, cnnf12, cnnf22]
cnn_char_csv_2 = [cnnf20, cnnf21, cnnf22]


# QCNNs

## Por caracteristicas

In [901]:
df_0_mean, df_0_std = metrics_comparison(char_csv_0, axis1)
df_0_mean = merge_metrics(df_0_mean)
df_0_std = merge_metrics(df_0_std)
# df_0.index = df_0.index.set_levels([256], level=0)

df_1_mean, df_1_std = metrics_comparison(char_csv_1, axis1)
df_1_mean = merge_metrics(df_1_mean)
df_1_std = merge_metrics(df_1_std)
# df_1.index = df_1.index.set_levels([512], level=0)

df_2_mean, df_2_std = metrics_comparison(char_csv_2, axis1)
df_2_mean = merge_metrics(df_2_mean)
df_2_std = merge_metrics(df_2_std)
# df_2.index = df_2.index.set_levels([1024], level=0)

df = pd.concat([df_0_mean, df_1_mean, df_2_mean])
df_std = pd.concat([df_0_std, df_1_std, df_2_std])

df = df.loc['test_acc']
df_std = df_std.loc['test_acc']

df.index = axis0
df_std.index = axis0


diffs = df.diff().loc[[512, 1024]]
diff_256_1024 = diffs.sum(numeric_only=True)

df_div = df_std/df
df_div = df_div.rename(index={256: '256_cv', 512: '512_cv', 1024: '1024_cv'})

df = pd.concat([df, df_std, df_div])

df.loc['diff_256_512'] = diffs.loc[512]
df.loc['diff_512_1024'] = diffs.loc[1024]
df.loc['diff_256_1024'] = diff_256_1024

stds = df.T.std(numeric_only=True)
means = df.T.mean(numeric_only=True)

df.loc[:, 'mean'] = means
df.loc[:, 'std'] = stds

In [902]:
df.sort_values('diff_256_512', axis=1)
df

df.loc[:, ['mean', 'std']]
print(df.loc[:, ['mean', 'std']].to_latex())

\begin{tabular}{lrr}
\toprule
{} &   mean &   std \\
{} \\
\midrule
256           &  0.719 & 0.103 \\
512           &  0.727 & 0.102 \\
1024          &  0.726 & 0.101 \\
256           &  0.033 & 0.028 \\
512           &  0.034 & 0.031 \\
1024          &  0.030 & 0.024 \\
256\_cv        &  0.048 & 0.043 \\
512\_cv        &  0.049 & 0.048 \\
1024\_cv       &  0.043 & 0.038 \\
diff\_256\_512  &  0.007 & 0.025 \\
diff\_512\_1024 & -0.000 & 0.024 \\
diff\_256\_1024 &  0.007 & 0.027 \\
\bottomrule
\end{tabular}



C:\Users\Master\AppData\Local\Temp\ipykernel_25148\429122676.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.loc[:, ['mean', 'std']].to_latex())


In [903]:
df_mean_part = df.groupby(level=1, axis=1).mean()

df_mean_part = pd.concat([df_mean_part, df_mean_part.iloc[3:6].loc[:, [1, 10, 20]]/df_mean_part.iloc[:3].loc[:, [1, 10, 20]]])
print(df_mean_part.iloc[12:].to_latex())
df_mean_part

\begin{tabular}{lrrrr}
\toprule
{} &     1 &    10 & \multicolumn{2}{l}{20} \\
\midrule
256  & 0.092 & 0.026 & 0.020 & NaN \\
512  & 0.099 & 0.023 & 0.019 & NaN \\
1024 & 0.079 & 0.026 & 0.019 & NaN \\
\bottomrule
\end{tabular}



C:\Users\Master\AppData\Local\Temp\ipykernel_25148\2313465068.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_mean_part.iloc[12:].to_latex())


,1,10,20,
256,0.698,0.730,0.731,0.411
512,0.700,0.741,0.739,0.414
1024,0.708,0.736,0.736,0.414
256,0.064,0.019,0.015,0.030
512,0.070,0.017,0.014,0.032
1024,0.056,0.019,0.014,0.027
256_cv,0.096,0.027,0.021,0.045
512_cv,0.104,0.023,0.020,0.048
1024_cv,0.083,0.027,0.020,0.041
diff_256_512,0.003,0.011,0.009,0.016


In [904]:
# print(df_part_mean.loc[:, [1, 10, 20]].to_latex())

In [905]:
df.groupby(level=0, axis=1).mean().sort_values('diff_256_512', axis=1, ascending=False)
# df.sort_values('diff_512_1024', axis=1, ascending=False)
# df.sort_values('diff_256_1024', axis=1, ascending=False)

,blues_vs_jazz,jazz_vs_pop,pop_vs_rock,country_vs_rock,country_vs_jazz,disco_vs_pop,disco_vs_hiphop,std,classical_vs_rock,reggae_vs_rock,...,country_vs_pop,jazz_vs_metal,disco_vs_rock,blues_vs_country,pop_vs_reggae,hiphop_vs_pop,classical_vs_hiphop,jazz_vs_rock,country_vs_hiphop,disco_vs_jazz
256,0.625,0.737,0.804,0.586,0.567,0.741,0.584,0.103,0.721,0.660,...,0.793,0.868,0.634,0.628,0.714,0.726,0.836,0.637,0.730,0.654
512,0.663,0.771,0.836,0.618,0.597,0.770,0.611,0.102,0.746,0.679,...,0.788,0.861,0.626,0.619,0.705,0.717,0.819,0.619,0.712,0.631
1024,0.637,0.753,0.826,0.611,0.586,0.768,0.615,0.101,0.766,0.665,...,0.792,0.875,0.622,0.644,0.712,0.699,0.843,0.640,0.716,0.656
256,0.042,0.042,0.022,0.029,0.039,0.047,0.029,0.028,0.029,0.027,...,0.035,0.024,0.048,0.025,0.018,0.035,0.015,0.021,0.034,0.051
512,0.027,0.039,0.022,0.034,0.038,0.042,0.030,0.031,0.042,0.034,...,0.030,0.019,0.037,0.042,0.045,0.041,0.041,0.043,0.034,0.030
1024,0.032,0.030,0.029,0.040,0.033,0.033,0.036,0.024,0.024,0.015,...,0.034,0.020,0.023,0.017,0.028,0.029,0.023,0.029,0.032,0.036
256_cv,0.072,0.058,0.028,0.054,0.075,0.065,0.051,0.043,0.041,0.042,...,0.044,0.028,0.075,0.040,0.026,0.049,0.018,0.033,0.046,0.081
512_cv,0.041,0.050,0.027,0.055,0.069,0.056,0.049,0.048,0.059,0.050,...,0.039,0.022,0.060,0.071,0.069,0.061,0.051,0.073,0.048,0.052
1024_cv,0.052,0.041,0.035,0.067,0.057,0.044,0.058,0.038,0.031,0.023,...,0.044,0.023,0.038,0.026,0.040,0.045,0.027,0.045,0.045,0.057
diff_256_512,0.038,0.034,0.033,0.032,0.030,0.029,0.027,0.025,0.025,0.019,...,-0.005,-0.007,-0.009,-0.009,-0.009,-0.009,-0.017,-0.018,-0.018,-0.023


In [906]:
df_red = df.loc[:, ['blues_vs_jazz', 'jazz_vs_pop', 'pop_vs_rock']]#, 'hiphop_vs_pop' , 'blues_vs_rock', 'disco_vs_reggae']]
means_red = df_red.T.mean(numeric_only=True)
# df_red.loc[:, 'mean'] = means_red

print(df_red.loc[[256, 512, 1024,'diff_256_512', 'diff_512_1024', '256_cv', '512_cv', '1024_cv'], :].to_latex())
df_red

\begin{tabular}{lrrrrrrrrr}
\toprule
{} & \multicolumn{3}{l}{blues\_vs\_jazz} & \multicolumn{3}{l}{jazz\_vs\_pop} & \multicolumn{3}{l}{pop\_vs\_rock} \\
{} &             1 &     10 &     20 &           1 &     10 &     20 &           1 &     10 &     20 \\
\midrule
256           &         0.560 &  0.654 &  0.660 &       0.690 &  0.753 &  0.767 &       0.777 &  0.818 &  0.816 \\
256           &         0.086 &  0.027 &  0.013 &       0.077 &  0.028 &  0.020 &       0.048 &  0.012 &  0.007 \\
512           &         0.647 &  0.675 &  0.667 &       0.760 &  0.779 &  0.774 &       0.847 &  0.831 &  0.832 \\
512           &         0.056 &  0.015 &  0.009 &       0.077 &  0.020 &  0.018 &       0.051 &  0.013 &  0.004 \\
1024          &         0.600 &  0.654 &  0.657 &       0.720 &  0.766 &  0.773 &       0.830 &  0.818 &  0.829 \\
1024          &         0.050 &  0.028 &  0.019 &       0.070 &  0.006 &  0.014 &       0.063 &  0.016 &  0.009 \\
diff\_256\_512  &         0.087 &  0.021 &  

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\58076611.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_red.loc[[256, 512, 1024,'diff_256_512', 'diff_512_1024', '256_cv', '512_cv', '1024_cv'], :].to_latex())


blues_vs_jazz               jazz_vs_pop                \
                          1     10     20           1     10     20   
256                   0.560  0.654  0.660       0.690  0.753  0.767   
512                   0.647  0.675  0.667       0.760  0.779  0.774   
1024                  0.600  0.654  0.657       0.720  0.766  0.773   
256                   0.086  0.027  0.013       0.077  0.028  0.020   
512                   0.056  0.015  0.009       0.077  0.020  0.018   
1024                  0.050  0.028  0.019       0.070  0.006  0.014   
256_cv                0.154  0.042  0.020       0.111  0.037  0.027   
512_cv                0.086  0.023  0.014       0.101  0.026  0.024   
1024_cv               0.083  0.043  0.029       0.097  0.008  0.019   
diff_256_512          0.087  0.021  0.007       0.070  0.026  0.007   
diff_512_1024        -0.047 -0.021 -0.010      -0.040 -0.013 -0.000   
diff_256_1024         0.040 -0.000 -0.003       0.030  0.013  0.007   

              pop_vs_rock                
                        1     10     20  
256                 0.777  0.818  0.816  
512                 0.847  0.831  0.832  
1024                0.830  0.818  0.829  
256                 0.048  0.012  0.007  
512                 0.051  0.013  0.004  
1024                0.063  0.016  0.009  
256_cv              0.062  0.015  0.009  
512_cv              0.060  0.015  0.005  
1024_cv             0.076  0.020  0.010  
diff_256_512        0.070  0.012  0.016  
diff_512_1024      -0.017 -0.013 -0.003  
diff_256_1024       0.053 -0.000  0.013

## Por muestras

In [907]:
axis0, axis1 = axis1, axis0

df_0_mean, df_0_std = metrics_comparison(muest_csv_0, axis1)
df_0_mean = merge_metrics(df_0_mean)
df_0_std = merge_metrics(df_0_std)

df_1_mean, df_1_std = metrics_comparison(muest_csv_1, axis1)
df_1_mean = merge_metrics(df_1_mean)
df_1_std = merge_metrics(df_1_std)

df_2_mean, df_2_std = metrics_comparison(muest_csv_2, axis1)
df_2_mean = merge_metrics(df_2_mean)
df_2_std = merge_metrics(df_2_std)

df = pd.concat([df_0_mean, df_1_mean, df_2_mean])
df_std = pd.concat([df_0_std, df_1_std, df_2_std])

df = df.loc['test_acc']
df_std = df_std.loc['test_acc']

df.index = axis0
df_std.index = axis0

diffs = df.diff().loc[[10, 20]]
diff_1_20 = diffs.sum(numeric_only=True)

df_div = df_std/df
df_div = df_div.rename(index={1: '1_cv', 10: '10_cv', 20: '20_cv'})

df = pd.concat([df, df_std, df_div])

df.loc['diff_1_10'] = diffs.loc[10]
df.loc['diff_10_20'] = diffs.loc[20]
df.loc['diff_1_20'] = diff_1_20

stds = df.T.std(numeric_only=True)
means = df.T.mean(numeric_only=True)

df.loc[:, 'mean'] = means
df.loc[:, 'std'] = stds

In [908]:
df.sort_values('diff_1_10', axis=1)
df

df.loc[:, ['mean', 'std']]
print(df.loc[:, ['mean', 'std']].to_latex())

\begin{tabular}{lrr}
\toprule
{} &   mean &   std \\
{} \\
\midrule
1          &  0.702 & 0.112 \\
10         &  0.736 & 0.095 \\
20         &  0.735 & 0.094 \\
1          &  0.063 & 0.026 \\
10         &  0.018 & 0.008 \\
20         &  0.014 & 0.006 \\
1\_cv       &  0.094 & 0.044 \\
10\_cv      &  0.026 & 0.012 \\
20\_cv      &  0.020 & 0.009 \\
diff\_1\_10  &  0.034 & 0.041 \\
diff\_10\_20 & -0.000 & 0.012 \\
diff\_1\_20  &  0.033 & 0.039 \\
\bottomrule
\end{tabular}



C:\Users\Master\AppData\Local\Temp\ipykernel_25148\2370816934.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.loc[:, ['mean', 'std']].to_latex())


In [909]:

df_part_mean = df.groupby(level=0, axis=1).mean().sort_values('diff_1_10', axis=1, ascending=False)
df_part_mean

# df

,country_vs_jazz,disco_vs_jazz,country_vs_disco,pop_vs_reggae,country_vs_reggae,blues_vs_rock,hiphop_vs_pop,blues_vs_disco,classical_vs_rock,disco_vs_reggae,...,blues_vs_metal,reggae_vs_rock,pop_vs_rock,metal_vs_pop,disco_vs_hiphop,hiphop_vs_rock,classical_vs_jazz,hiphop_vs_jazz,classical_vs_hiphop,classical_vs_pop
1,0.517,0.589,0.611,0.667,0.600,0.527,0.663,0.592,0.703,0.530,...,0.767,0.659,0.818,0.958,0.604,0.708,0.707,0.689,0.859,0.891
10,0.621,0.679,0.692,0.742,0.674,0.600,0.736,0.657,0.768,0.591,...,0.773,0.665,0.822,0.960,0.598,0.699,0.693,0.670,0.825,0.857
20,0.612,0.674,0.683,0.723,0.673,0.610,0.742,0.641,0.760,0.602,...,0.776,0.680,0.825,0.954,0.607,0.710,0.713,0.670,0.815,0.851
1,0.065,0.081,0.069,0.063,0.076,0.043,0.076,0.053,0.061,0.074,...,0.061,0.043,0.054,0.040,0.066,0.066,0.056,0.070,0.057,0.039
10,0.025,0.017,0.021,0.018,0.012,0.015,0.017,0.021,0.019,0.025,...,0.029,0.017,0.014,0.012,0.016,0.024,0.016,0.013,0.012,0.014
20,0.020,0.019,0.019,0.011,0.017,0.011,0.013,0.014,0.015,0.012,...,0.020,0.017,0.007,0.009,0.013,0.013,0.013,0.014,0.010,0.007
1_cv,0.127,0.136,0.115,0.096,0.126,0.083,0.114,0.088,0.087,0.140,...,0.080,0.066,0.066,0.042,0.109,0.092,0.079,0.102,0.069,0.043
10_cv,0.041,0.025,0.031,0.024,0.018,0.026,0.023,0.031,0.024,0.043,...,0.037,0.025,0.016,0.012,0.027,0.034,0.023,0.019,0.015,0.016
20_cv,0.033,0.029,0.028,0.015,0.025,0.019,0.018,0.021,0.020,0.020,...,0.025,0.025,0.008,0.009,0.022,0.018,0.018,0.021,0.013,0.008
diff_1_10,0.105,0.090,0.081,0.075,0.074,0.074,0.072,0.065,0.065,0.061,...,0.006,0.006,0.005,0.002,-0.006,-0.009,-0.013,-0.019,-0.034,-0.035


In [910]:
# print(df_part_mean.loc[:, [256, 512, 1024]].to_latex())

In [911]:
df.sort_values('diff_1_10', axis=1, ascending=False)
# df.sort_values('diff_10_20', axis=1, ascending=False)
# df.sort_values('1_cv', axis=1)
# df.sort_values('10_cv', axis=1)

disco_vs_jazz country_vs_disco country_vs_jazz         \
                    512              256             512    256    
1                  0.530            0.567           0.517  0.487   
10                 0.685            0.695           0.644  0.611   
20                 0.678            0.686           0.630  0.603   
1                  0.058            0.096           0.074  0.080   
10                 0.013            0.023           0.031  0.017   
20                 0.017            0.018           0.010  0.019   
1_cv               0.110            0.170           0.142  0.165   
10_cv              0.020            0.032           0.048  0.028   
20_cv              0.025            0.026           0.016  0.031   
diff_1_10          0.155            0.129           0.128  0.125   
diff_10_20        -0.008           -0.009          -0.014 -0.009   
diff_1_20          0.148            0.119           0.113  0.116   

           hiphop_vs_pop pop_vs_reggae disco_vs_reggae country_vs_rock  \
                    1024          512             1024            256    
1                  0.613         0.637           0.483           0.523   
10                 0.736         0.755           0.597           0.627   
20                 0.747         0.724           0.597           0.609   
1                  0.059         0.110           0.077           0.064   
10                 0.013         0.013           0.024           0.011   
20                 0.015         0.012           0.011           0.013   
1_cv               0.097         0.172           0.160           0.123   
10_cv              0.017         0.017           0.039           0.017   
20_cv              0.020         0.017           0.018           0.022   
diff_1_10          0.122         0.118           0.114           0.103   
diff_10_20         0.011        -0.031           0.000          -0.017   
diff_1_20          0.134         0.088           0.114           0.086   

           classical_vs_rock blues_vs_rock  ... blues_vs_metal  \
                        256           512   ...           256    
1                      0.653         0.503  ...          0.783   
10                     0.756         0.605  ...          0.760   
20                     0.752         0.626  ...          0.777   
1                      0.049         0.049  ...          0.033   
10                     0.018         0.011  ...          0.029   
20                     0.019         0.008  ...          0.021   
1_cv                   0.075         0.098  ...          0.043   
10_cv                  0.023         0.018  ...          0.038   
20_cv                  0.026         0.013  ...          0.027   
diff_1_10              0.103         0.102  ...         -0.023   
diff_10_20            -0.005         0.021  ...          0.016   
diff_1_20              0.098         0.122  ...         -0.007   

           classical_vs_jazz classical_vs_pop country_vs_hiphop  \
                        256              1024              256    
1                      0.710            0.887             0.753   
10                     0.683            0.859             0.720   
20                     0.708            0.854             0.717   
1                      0.067            0.059             0.067   
10                     0.011            0.011             0.019   
20                     0.016            0.008             0.015   
1_cv                   0.095            0.067             0.089   
10_cv                  0.016            0.013             0.026   
20_cv                  0.022            0.009             0.021   
diff_1_10             -0.027           -0.028            -0.033   
diff_10_20             0.024           -0.005            -0.003   
diff_1_20             -0.002           -0.032            -0.036   

           classical_vs_pop        hiphop_vs_jazz classical_vs_hiphop  \
                       256    512            256                 256    
1          

In [912]:
df_red = df.loc[:, ['disco_vs_jazz', 'country_vs_disco', 'country_vs_jazz']]#, 'hiphop_vs_pop' , 'blues_vs_rock', 'disco_vs_reggae']]
means_red = df_red.T.mean(numeric_only=True)
# df_red.loc[:, 'mean'] = means_red

print(df_red.loc[[1, 10, 20, '1_cv', '10_cv', '20_cv', 'diff_1_10', 'diff_10_20'], :].to_latex())
df_red

\begin{tabular}{lrrrrrrrrr}
\toprule
{} & \multicolumn{3}{l}{disco\_vs\_jazz} & \multicolumn{3}{l}{country\_vs\_disco} & \multicolumn{3}{l}{country\_vs\_jazz} \\
{} &           256 &    512 &  1024 &              256 &    512 &   1024 &             256 &    512 &   1024 \\
\midrule
1          &         0.610 &  0.530 & 0.627 &            0.567 &  0.607 &  0.660 &           0.487 &  0.517 &  0.547 \\
1          &         0.106 &  0.058 & 0.077 &            0.096 &  0.058 &  0.052 &           0.080 &  0.074 &  0.040 \\
10         &         0.683 &  0.685 & 0.668 &            0.695 &  0.690 &  0.689 &           0.611 &  0.644 &  0.608 \\
10         &         0.020 &  0.013 & 0.017 &            0.023 &  0.027 &  0.014 &           0.017 &  0.031 &  0.028 \\
20         &         0.670 &  0.678 & 0.673 &            0.686 &  0.687 &  0.676 &           0.603 &  0.630 &  0.604 \\
20         &         0.025 &  0.017 & 0.016 &            0.018 &  0.022 &  0.018 &           0.019 &  0.010 &  0.032 

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\2575052748.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_red.loc[[1, 10, 20, '1_cv', '10_cv', '20_cv', 'diff_1_10', 'diff_10_20'], :].to_latex())


disco_vs_jazz              country_vs_disco                \
                     256    512  1024              256    512   1024   
1                  0.610  0.530 0.627            0.567  0.607  0.660   
10                 0.683  0.685 0.668            0.695  0.690  0.689   
20                 0.670  0.678 0.673            0.686  0.687  0.676   
1                  0.106  0.058 0.077            0.096  0.058  0.052   
10                 0.020  0.013 0.017            0.023  0.027  0.014   
20                 0.025  0.017 0.016            0.018  0.022  0.018   
1_cv               0.175  0.110 0.123            0.170  0.096  0.079   
10_cv              0.030  0.020 0.025            0.032  0.040  0.020   
20_cv              0.037  0.025 0.023            0.026  0.033  0.026   
diff_1_10          0.073  0.155 0.041            0.129  0.084  0.029   
diff_10_20        -0.013 -0.008 0.006           -0.009 -0.003 -0.014   
diff_1_20          0.060  0.148 0.047            0.119  0.081  0.015   

           country_vs_jazz                
                       256    512   1024  
1                    0.487  0.517  0.547  
10                   0.611  0.644  0.608  
20                   0.603  0.630  0.604  
1                    0.080  0.074  0.040  
10                   0.017  0.031  0.028  
20                   0.019  0.010  0.032  
1_cv                 0.165  0.142  0.073  
10_cv                0.028  0.048  0.046  
20_cv                0.031  0.016  0.053  
diff_1_10            0.125  0.128  0.062  
diff_10_20          -0.009 -0.014 -0.004  
diff_1_20            0.116  0.113  0.058

# CNNs

## Por caracteristicas

In [913]:
axis0, axis1 = axis1, axis0

df_0_mean, df_0_std = metrics_comparison(cnn_char_csv_0, axis1)
df_0_mean = merge_metrics(df_0_mean)
df_0_std = merge_metrics(df_0_std)
# df_0.index = df_0.index.set_levels([256], level=0)

df_1_mean, df_1_std = metrics_comparison(cnn_char_csv_1, axis1)
df_1_mean = merge_metrics(df_1_mean)
df_1_std = merge_metrics(df_1_std)
# df_1.index = df_1.index.set_levels([512], level=0)

df_2_mean, df_2_std = metrics_comparison(cnn_char_csv_2, axis1)
df_2_mean = merge_metrics(df_2_mean)
df_2_std = merge_metrics(df_2_std)
# df_2.index = df_2.index.set_levels([1024], level=0)

df = pd.concat([df_0_mean, df_1_mean, df_2_mean])
df_std = pd.concat([df_0_std, df_1_std, df_2_std])

df = df.loc['test_acc']
df_std = df_std.loc['test_acc']

df.index = axis0
df_std.index = axis0

diffs = df.diff().loc[[512, 1024]]
diff_256_1024 = diffs.sum(numeric_only=True)

df_div = df_std/df
df_div = df_div.rename(index={256: '256_cv', 512: '512_cv', 1024: '1024_cv'})

df = pd.concat([df, df_std, df_div])

df.loc['diff_256_512'] = diffs.loc[512]
df.loc['diff_512_1024'] = diffs.loc[1024]
df.loc['diff_256_1024'] = diff_256_1024


stds = df.T.std(numeric_only=True)
means = df.T.mean(numeric_only=True)

df.loc[:, 'mean'] = means
df.loc[:, 'std'] = stds

In [914]:
df.groupby(level=1, axis=1).mean()
print(df.groupby(level=1, axis=1).mean().iloc[:-3].to_latex())
df.groupby(level=1, axis=1).mean().iloc[:-3]

\begin{tabular}{lrrrr}
\toprule
{} &     1 &    10 & \multicolumn{2}{l}{20} \\
\midrule
256     & 0.705 & 0.787 & 0.798 & 0.432 \\
512     & 0.717 & 0.796 & 0.815 & 0.440 \\
1024    & 0.719 & 0.791 & 0.809 & 0.437 \\
256     & 0.072 & 0.029 & 0.034 & 0.042 \\
512     & 0.061 & 0.034 & 0.022 & 0.035 \\
1024    & 0.071 & 0.034 & 0.025 & 0.040 \\
256\_cv  & 0.104 & 0.037 & 0.044 & 0.058 \\
512\_cv  & 0.088 & 0.045 & 0.028 & 0.050 \\
1024\_cv & 0.101 & 0.045 & 0.032 & 0.056 \\
\bottomrule
\end{tabular}



C:\Users\Master\AppData\Local\Temp\ipykernel_25148\995343046.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.groupby(level=1, axis=1).mean().iloc[:-3].to_latex())


,1,10,20,
256,0.705,0.787,0.798,0.432
512,0.717,0.796,0.815,0.440
1024,0.719,0.791,0.809,0.437
256,0.072,0.029,0.034,0.042
512,0.061,0.034,0.022,0.035
1024,0.071,0.034,0.025,0.040
256_cv,0.104,0.037,0.044,0.058
512_cv,0.088,0.045,0.028,0.050
1024_cv,0.101,0.045,0.032,0.056


In [915]:
df.sort_values('diff_256_512', axis=1)
df

df.loc[:, ['mean', 'std']]
print(df.loc[:, ['mean', 'std']].to_latex())

\begin{tabular}{lrr}
\toprule
{} &   mean &   std \\
{} \\
\midrule
256           &  0.763 & 0.102 \\
512           &  0.776 & 0.103 \\
1024          &  0.773 & 0.100 \\
256           &  0.045 & 0.040 \\
512           &  0.039 & 0.031 \\
1024          &  0.043 & 0.036 \\
256\_cv        &  0.062 & 0.055 \\
512\_cv        &  0.054 & 0.046 \\
1024\_cv       &  0.060 & 0.053 \\
diff\_256\_512  &  0.013 & 0.035 \\
diff\_512\_1024 & -0.003 & 0.033 \\
diff\_256\_1024 &  0.010 & 0.040 \\
\bottomrule
\end{tabular}



C:\Users\Master\AppData\Local\Temp\ipykernel_25148\429122676.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.loc[:, ['mean', 'std']].to_latex())


In [927]:
df
df_part_mean = df.groupby(level=0, axis=1).mean()
df_part_mean.sort_values('diff_256_512', axis=1, ascending=False)
df_part_mean

print(df.loc[:, ['blues_vs_classical', 'blues_vs_country', 'blues_vs_disco']].to_latex())

\begin{tabular}{lrrrrrrrrr}
\toprule
{} & \multicolumn{3}{l}{blues\_vs\_classical} & \multicolumn{3}{l}{blues\_vs\_country} & \multicolumn{3}{l}{blues\_vs\_disco} \\
{} &                  1 &    10 &    20 &                1 &    10 &    20 &              1 &    10 &    20 \\
\midrule
256           &              78.0\% & 76.1\% & 85.6\% &            59.0\% & 70.2\% & 73.8\% &          62.7\% & 72.4\% & 71.9\% \\
512           &              71.0\% & 88.0\% & 88.8\% &            66.3\% & 72.0\% & 77.5\% &          61.0\% & 73.6\% & 75.4\% \\
1024          &              78.7\% & 86.8\% & 87.4\% &            67.0\% & 74.9\% & 74.3\% &          63.3\% & 71.6\% & 76.8\% \\
256           &               5.2\% & 13.2\% &  3.5\% &             6.3\% &  3.4\% &  4.0\% &           9.0\% &  5.4\% & 12.9\% \\
512           &               6.1\% &  2.3\% &  1.6\% &             3.2\% &  3.5\% &  1.3\% &           5.8\% &  3.0\% &  3.4\% \\
1024          &               4.6\% &  2.0\% &  2.0\% &    

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\2421295562.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.loc[:, ['blues_vs_classical', 'blues_vs_country', 'blues_vs_disco']].to_latex())


In [849]:
print(df_part_mean.loc[:, [1, 10, 20]].to_latex())

KeyError: "None of [Int64Index([1, 10, 20], dtype='int64')] are in the [columns]"

In [ ]:
df.sort_values('diff_256_512', axis=1, ascending=False)
df.sort_values('diff_512_1024', axis=1, ascending=False)
df.sort_values('diff_256_1024', axis=1, ascending=False)

jazz_vs_pop blues_vs_metal metal_vs_reggae blues_vs_classical  \
                       1              1               1                  10   
256                 0.613          0.717           0.797              0.761   
512                 0.700          0.783           0.890              0.880   
1024                0.777          0.847           0.913              0.868   
256_cv              0.195          0.116           0.230              0.174   
512_cv              0.108          0.098           0.037              0.026   
1024_cv             0.056          0.038           0.027              0.023   
diff_256_512        0.087          0.067           0.093              0.119   
diff_512_1024       0.077          0.063           0.023             -0.012   
diff_256_1024       0.163          0.130           0.117              0.107   

              blues_vs_jazz classical_vs_metal        jazz_vs_reggae  \
                         1                  20     1              20   
256                   0.577              0.879  0.840          0.702   
512                   0.653              0.970  0.957          0.789   
1024                  0.680              0.977  0.930          0.788   
256_cv                0.044              0.244  0.263          0.170   
512_cv                0.087              0.009  0.034          0.018   
1024_cv               0.112              0.005  0.029          0.025   
diff_256_512          0.077              0.091  0.117          0.087   
diff_512_1024         0.027              0.007 -0.027         -0.001   
diff_256_1024         0.103              0.098  0.090          0.086   

              blues_vs_country disco_vs_pop  ... country_vs_pop  \
                            1            10  ...             1    
256                      0.590        0.727  ...          0.750   
512                      0.663        0.797  ...          0.740   
1024                     0.670        0.801  ...          0.710   
256_cv                   0.107        0.085  ...          0.132   
512_cv                   0.048        0.027  ...          0.116   
1024_cv                  0.062        0.034  ...          0.154   
diff_256_512             0.073        0.070  ...         -0.010   
diff_512_1024            0.007        0.004  ...         -0.030   
diff_256_1024            0.080        0.074  ...         -0.040   

              classical_vs_rock hiphop_vs_jazz reggae_vs_rock blues_vs_reggae  \
                             20             20             1               10   
256                       0.891          0.789          0.620           0.756   
512                       0.889          0.818          0.607           0.759   
1024                      0.851          0.748          0.577           0.708   
256_cv                    0.005          0.026          0.162           0.034   
512_cv                    0.014          0.030          0.084           0.027   
1024_cv                   0.052          0.195          0.186           0.107   
diff_256_512             -0.002          0.029         -0.013           0.003   
diff_512_1024            -0.038         -0.070         -0.030          -0.052   
diff_256_1024            -0.040         -0.041         -0.043          -0.049   

              blues_vs_pop disco_vs_reggae blues_vs_hiphop disco_vs_metal  \
                        20              10              1              1    
256                  0.964           0.682           0.700          0.823   
512                  0.963           0.672           0.680          0.797   
1024                 0.914           0.629           0.630          0.747   
256_cv               0.010           0.005           0.073          0.057   
512_cv               0.012           0.033           0.097          0.018   
1024_cv              0.039           0.075           0.185          0.211   
diff_256_512        -0.001          -0.010          -0.020         -0.027   
diff_512_1024       -0.049  

## Por muestras

In [850]:
axis0, axis1 = axis1, axis0

df_0_mean, df_0_std = metrics_comparison(cnn_muest_csv_0, axis1)
df_0_mean = merge_metrics(df_0_mean)
df_0_std = merge_metrics(df_0_std)
# df_0.index = df_0.index.set_levels([256], level=0)

df_1_mean, df_1_std = metrics_comparison(cnn_muest_csv_1, axis1)
df_1_mean = merge_metrics(df_1_mean)
df_1_std = merge_metrics(df_1_std)
# df_1.index = df_1.index.set_levels([512], level=0)

df_2_mean, df_2_std = metrics_comparison(cnn_muest_csv_2, axis1)
df_2_mean = merge_metrics(df_2_mean)
df_2_std = merge_metrics(df_2_std)
# df_2.index = df_2.index.set_levels([1024], level=0)

df = pd.concat([df_0_mean, df_1_mean, df_2_mean])
df_std = pd.concat([df_0_std, df_1_std, df_2_std])

df = df.loc['test_acc']
df_std = df_std.loc['test_acc']

df.index = axis0
df_std.index = axis0

diffs = df.diff().loc[[10, 20]]
diff_1_20 = diffs.sum(numeric_only=True)

df_div = df_std/df
df_div = df_div.rename(index={1: '1_cv', 10: '10_cv', 20: '20_cv'})

df = pd.concat([df, df_div])

df.loc['diff_1_10'] = diffs.loc[10]
df.loc['diff_10_20'] = diffs.loc[20]
df.loc['diff_1_20'] = diff_1_20

stds = df.T.std(numeric_only=True)
means = df.T.mean(numeric_only=True)

cv = stds/means

df.loc[:, 'mean'] = means
df.loc[:, 'std'] = stds
df.loc[:, 'cv'] = cv

In [865]:
df_part_mean = df.groupby(level=0, axis=1).mean()
df_part_mean.sort_values('diff_1_10', axis=1, ascending=True)
# df.loc[:, ['mean', 'std']]

# print(df.loc[:, ['mean', 'std']].to_latex())

print(df.loc[:, ['blues_vs_rock', 'jazz_vs_pop', 'disco_vs_pop']].to_latex())
df.loc[:, ['blues_vs_rock', 'jazz_vs_pop', 'disco_vs_pop']]

\begin{tabular}{lrrrrrrrrr}
\toprule
{} & \multicolumn{3}{l}{blues\_vs\_rock} & \multicolumn{3}{l}{jazz\_vs\_pop} & \multicolumn{3}{l}{disco\_vs\_pop} \\
{} &           256 &   512 &  1024 &         256 &   512 &  1024 &          256 &   512 &  1024 \\
\midrule
1          &         55.3\% & 56.0\% & 58.7\% &       61.3\% & 70.0\% & 77.7\% &        61.0\% & 67.3\% & 66.3\% \\
10         &         72.5\% & 71.7\% & 70.5\% &       81.1\% & 84.5\% & 84.2\% &        72.7\% & 79.7\% & 80.1\% \\
20         &         73.5\% & 74.8\% & 71.4\% &       80.0\% & 85.3\% & 85.7\% &        78.5\% & 78.3\% & 80.5\% \\
1\_cv       &         14.5\% & 13.2\% & 10.1\% &       19.5\% & 10.8\% &  5.6\% &        17.1\% & 12.2\% & 13.6\% \\
10\_cv      &          3.4\% &  4.8\% &  6.2\% &        4.7\% &  4.6\% &  2.2\% &         8.5\% &  2.7\% &  3.4\% \\
20\_cv      &          5.2\% &  1.0\% &  6.0\% &        7.8\% &  2.7\% &  1.4\% &         2.0\% &  7.6\% &  1.3\% \\
diff\_1\_10  &         17.2\% & 15.7\% 

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\3391355622.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.loc[:, ['blues_vs_rock', 'jazz_vs_pop', 'disco_vs_pop']].to_latex())


blues_vs_rock             jazz_vs_pop             disco_vs_pop  \
                     256   512  1024         256   512  1024          256   
1                  55.3% 56.0% 58.7%       61.3% 70.0% 77.7%        61.0%   
10                 72.5% 71.7% 70.5%       81.1% 84.5% 84.2%        72.7%   
20                 73.5% 74.8% 71.4%       80.0% 85.3% 85.7%        78.5%   
1_cv               14.5% 13.2% 10.1%       19.5% 10.8%  5.6%        17.1%   
10_cv               3.4%  4.8%  6.2%        4.7%  4.6%  2.2%         8.5%   
20_cv               5.2%  1.0%  6.0%        7.8%  2.7%  1.4%         2.0%   
diff_1_10          17.2% 15.7% 11.8%       19.8% 14.5%  6.5%        11.7%   
diff_10_20          1.0%  3.1%  0.9%       -1.1%  0.8%  1.5%         5.9%   
diff_1_20          18.1% 18.8% 12.7%       18.6% 15.3%  8.0%        17.5%   

                        
             512  1024  
1          67.3% 66.3%  
10         79.7% 80.1%  
20         78.3% 80.5%  
1_cv       12.2% 13.6%  
10_cv       2.7%  3.4%  
20_cv       7.6%  1.3%  
diff_1_10  12.3% 13.8%  
diff_10_20 -1.4%  0.4%  
diff_1_20  10.9% 14.2%

In [629]:
print(df_part_mean.loc[:, [256, 512, 1024]].to_latex())

\begin{tabular}{lrrr}
\toprule
{} &   256 &   512 &  1024 \\
\midrule
1          & 0.705 & 0.717 & 0.719 \\
10         & 0.787 & 0.796 & 0.791 \\
20         & 0.798 & 0.815 & 0.809 \\
1\_cv       & 0.104 & 0.088 & 0.101 \\
10\_cv      & 0.037 & 0.045 & 0.045 \\
20\_cv      & 0.044 & 0.028 & 0.032 \\
diff\_1\_10  & 0.083 & 0.079 & 0.071 \\
diff\_10\_20 & 0.011 & 0.019 & 0.018 \\
diff\_1\_20  & 0.093 & 0.098 & 0.090 \\
\bottomrule
\end{tabular}



C:\Users\Master\AppData\Local\Temp\ipykernel_25148\1709105800.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_part_mean.loc[:, [256, 512, 1024]].to_latex())


In [630]:
df.sort_values('diff_1_10', axis=1, ascending=False)
df.sort_values('diff_10_20', axis=1, ascending=False)
# df.sort_values('diff_1_20', axis=1, ascending=False)

,cv,blues_vs_classical,blues_vs_pop,reggae_vs_rock,country_vs_pop,disco_vs_hiphop,blues_vs_reggae,metal_vs_rock,blues_vs_jazz,classical_vs_jazz,...,pop_vs_reggae,hiphop_vs_pop,country_vs_rock,hiphop_vs_pop,jazz_vs_reggae,hiphop_vs_jazz,blues_vs_jazz,blues_vs_reggae,country_vs_reggae,classical_vs_metal
,,256,512,512,512,1024,1024,512,1024,1024,...,256,1024,1024,512,256,1024,512,256,512,256
1,0.148,0.780,0.813,0.607,0.740,0.633,0.660,0.757,0.680,0.753,...,0.670,0.747,0.607,0.663,0.650,0.710,0.653,0.650,0.630,0.840
10,0.112,0.761,0.878,0.662,0.766,0.642,0.708,0.753,0.731,0.757,...,0.764,0.787,0.688,0.810,0.729,0.777,0.735,0.756,0.742,0.964
20,0.105,0.856,0.963,0.741,0.844,0.707,0.772,0.815,0.793,0.817,...,0.750,0.770,0.668,0.789,0.702,0.748,0.704,0.723,0.686,0.879
1_cv,0.519,0.067,0.022,0.084,0.116,0.135,0.102,0.082,0.112,0.109,...,0.138,0.142,0.074,0.175,0.079,0.027,0.087,0.172,0.098,0.263
10_cv,0.853,0.174,0.057,0.153,0.197,0.147,0.107,0.126,0.173,0.097,...,0.025,0.011,0.028,0.028,0.036,0.064,0.194,0.034,0.023,0.007
20_cv,1.158,0.041,0.012,0.021,0.017,0.024,0.058,0.019,0.011,0.013,...,0.096,0.041,0.048,0.043,0.170,0.195,0.138,0.123,0.177,0.244
diff_1_10,0.570,-0.019,0.064,0.055,0.026,0.009,0.048,-0.004,0.051,0.003,...,0.094,0.041,0.081,0.147,0.079,0.067,0.081,0.106,0.112,0.124
diff_10_20,1.602,0.095,0.085,0.079,0.078,0.065,0.064,0.062,0.062,0.061,...,-0.014,-0.018,-0.020,-0.021,-0.027,-0.029,-0.031,-0.033,-0.055,-0.086
diff_1_20,0.466,0.076,0.150,0.134,0.104,0.074,0.112,0.058,0.113,0.064,...,0.080,0.023,0.062,0.126,0.052,0.038,0.050,0.073,0.056,0.038


# Por generos (QCNN)

In [631]:
import itertools

axis1 = [1, 10, 20]
axis0 = [256, 512, 1024]

df = pd.DataFrame()

df_multiindex = pd.MultiIndex.from_product([axis0, axis1])
df.index = df_multiindex

matrix = [char_csv_0, char_csv_1, char_csv_2]
indices = list(itertools.product(axis0, axis1))

indices = [indices[:3], indices[3:6], indices[6:]]

# print(list(zip(matrix, indices)))
# print(indices)

qcnn_csv = {}

axis1 = [1, 10, 20]
axis0 = [256, 512, 1024]

for i, ch in enumerate(axis0):
    row = {}
    for j, m in enumerate(axis1):
        row[m] = matrix[i][j]
    qcnn_csv[ch] = row

# print(qcnn_csv)

# print(qcnn_csv)
# qcnn_csv = {{m: e for m} for }

# qcnn_csv = char_csv_0
# qcnn_csv.extend(char_csv_1)
# qcnn_csv.extend(char_csv_2)

frow = metrics_genres(qcnn_csv[256][1])

df = pd.DataFrame(index=df_multiindex, columns=frow.index)

for ch in qcnn_csv:
    for m in qcnn_csv[ch]:
        row = metrics_genres(qcnn_csv[ch][m]).loc['mean']
        df.loc[ch].loc[m] = row
        

# print(df.loc['mean'].to_frame().T.to_latex())
print(df.iloc[:, :-2].to_latex())
df.iloc[:, :-2]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\1443399483.py:50: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.iloc[:, :-2].to_latex())


\begin{tabular}{llllllllllll}
\toprule
     &    & blues & classical & country & disco & hiphop &  jazz & metal &   pop & reggae &  rock \\
\midrule
256  & 1  & 0.669 &     0.774 &   0.640 & 0.635 &  0.704 & 0.644 & 0.823 & 0.785 &  0.655 & 0.646 \\
     & 10 & 0.705 &     0.789 &   0.703 & 0.690 &  0.703 & 0.688 & 0.847 & 0.804 &  0.684 & 0.687 \\
     & 20 & 0.704 &     0.787 &   0.697 & 0.684 &  0.705 & 0.691 & 0.850 & 0.805 &  0.691 & 0.691 \\
512  & 1  & 0.672 &     0.776 &   0.644 & 0.646 &  0.689 & 0.648 & 0.827 & 0.788 &  0.652 & 0.660 \\
     & 10 & 0.720 &     0.799 &   0.713 & 0.696 &  0.715 & 0.699 & 0.851 & 0.822 &  0.702 & 0.694 \\
     & 20 & 0.715 &     0.794 &   0.707 & 0.695 &  0.712 & 0.700 & 0.854 & 0.817 &  0.699 & 0.700 \\
1024 & 1  & 0.670 &     0.783 &   0.670 & 0.654 &  0.698 & 0.664 & 0.831 & 0.783 &  0.654 & 0.670 \\
     & 10 & 0.711 &     0.798 &   0.708 & 0.688 &  0.707 & 0.696 & 0.851 & 0.813 &  0.697 & 0.690 \\
     & 20 & 0.707 &     0.796 &   0.700 & 0

blues classical country disco hiphop  jazz metal   pop reggae  rock
256  1  0.669     0.774   0.640 0.635  0.704 0.644 0.823 0.785  0.655 0.646
     10 0.705     0.789   0.703 0.690  0.703 0.688 0.847 0.804  0.684 0.687
     20 0.704     0.787   0.697 0.684  0.705 0.691 0.850 0.805  0.691 0.691
512  1  0.672     0.776   0.644 0.646  0.689 0.648 0.827 0.788  0.652 0.660
     10 0.720     0.799   0.713 0.696  0.715 0.699 0.851 0.822  0.702 0.694
     20 0.715     0.794   0.707 0.695  0.712 0.700 0.854 0.817  0.699 0.700
1024 1  0.670     0.783   0.670 0.654  0.698 0.664 0.831 0.783  0.654 0.670
     10 0.711     0.798   0.708 0.688  0.707 0.696 0.851 0.813  0.697 0.690
     20 0.707     0.796   0.700 0.689  0.713 0.697 0.855 0.813  0.694 0.693

# Por generos (CNN)

In [770]:
import itertools

df = pd.DataFrame()

df_multiindex = pd.MultiIndex.from_product([axis0, axis1])
df.index = df_multiindex

matrix = [cnn_char_csv_0, cnn_char_csv_1, cnn_char_csv_2]
indices = list(itertools.product(axis0, axis1))

indices = [indices[:3], indices[3:6], indices[6:]]

# print(list(zip(matrix, indices)))
# print(indices)

cnn_csv = {}

axis1 = [1, 10, 20]
axis0 = [256, 512, 1024]

for i, ch in enumerate(axis0):
    row = {}
    for j, m in enumerate(axis1):
        row[m] = matrix[i][j]
    cnn_csv[ch] = row

# print(qcnn_csv)

# print(qcnn_csv)
# qcnn_csv = {{m: e for m} for }

# qcnn_csv = char_csv_0
# qcnn_csv.extend(char_csv_1)
# qcnn_csv.extend(char_csv_2)

frow = metrics_genres(cnn_csv[256][1])

df = pd.DataFrame(index=df_multiindex, columns=frow.index)

for ch in cnn_csv:
    for m in cnn_csv[ch]:
        row = metrics_genres(cnn_csv[ch][m]).loc['mean']
        df.loc[ch].loc[m] = row
        

# print(df.loc['mean'].to_frame().T.to_latex())
print(df.iloc[:, :-2].to_latex())
df.iloc[:, :-2]

\begin{tabular}{llllllllllll}
\toprule
     &    & blues & classical & country & disco & hiphop &  jazz & metal &   pop & reggae &  rock \\
\midrule
256  & 1  & 67.5\% &     79.7\% &   66.6\% & 64.1\% &  69.7\% & 66.7\% & 82.4\% & 75.4\% &  67.0\% & 65.5\% \\
     & 10 & 77.6\% &     85.0\% &   75.5\% & 73.1\% &  76.8\% & 76.6\% & 89.3\% & 83.4\% &  75.4\% & 74.5\% \\
     & 20 & 79.3\% &     86.1\% &   76.8\% & 74.7\% &  78.1\% & 77.5\% & 88.9\% & 84.8\% &  74.9\% & 76.6\% \\
512  & 1  & 68.3\% &     80.9\% &   67.0\% & 64.9\% &  70.1\% & 70.8\% & 85.3\% & 75.3\% &  69.3\% & 65.4\% \\
     & 10 & 78.4\% &     87.7\% &   76.2\% & 74.2\% &  77.6\% & 79.0\% & 89.0\% & 84.7\% &  75.4\% & 73.8\% \\
     & 20 & 80.8\% &     89.0\% &   78.2\% & 76.5\% &  78.8\% & 80.1\% & 90.5\% & 86.6\% &  77.3\% & 77.5\% \\
1024 & 1  & 70.6\% &     81.0\% &   67.0\% & 65.3\% &  69.7\% & 69.6\% & 83.8\% & 77.9\% &  67.7\% & 66.8\% \\
     & 10 & 78.1\% &     86.7\% &   76.9\% & 73.4\% &  76.6\% & 77.7\% & 8

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\1002400502.py:47: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.iloc[:, :-2].to_latex())


blues classical country disco hiphop  jazz metal   pop reggae  rock
256  1  67.5%     79.7%   66.6% 64.1%  69.7% 66.7% 82.4% 75.4%  67.0% 65.5%
     10 77.6%     85.0%   75.5% 73.1%  76.8% 76.6% 89.3% 83.4%  75.4% 74.5%
     20 79.3%     86.1%   76.8% 74.7%  78.1% 77.5% 88.9% 84.8%  74.9% 76.6%
512  1  68.3%     80.9%   67.0% 64.9%  70.1% 70.8% 85.3% 75.3%  69.3% 65.4%
     10 78.4%     87.7%   76.2% 74.2%  77.6% 79.0% 89.0% 84.7%  75.4% 73.8%
     20 80.8%     89.0%   78.2% 76.5%  78.8% 80.1% 90.5% 86.6%  77.3% 77.5%
1024 1  70.6%     81.0%   67.0% 65.3%  69.7% 69.6% 83.8% 77.9%  67.7% 66.8%
     10 78.1%     86.7%   76.9% 73.4%  76.6% 77.7% 88.5% 84.8%  73.5% 74.5%
     20 80.4%     88.4%   77.6% 76.5%  77.6% 79.9% 90.5% 85.6%  76.6% 76.0%

# Metricas QCNN

In [633]:
filename = "4782-0-0.csv"
df = return_metrics(filename)
df.loc["Mean"]


C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.698
             std    0.064
train_acc    mean   0.741
             std    0.031
mean_vs_std         0.096
Name: Mean, dtype: object

In [634]:
filename = "4783-0-1.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.730
             std    0.019
train_acc    mean   0.735
             std    0.012
mean_vs_std         0.027
Name: Mean, dtype: object

In [635]:
filename = "4829-0-2.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.731
             std    0.015
train_acc    mean   0.732
             std    0.010
mean_vs_std         0.021
Name: Mean, dtype: object

In [636]:
filename = "4849-1-0.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.700
             std    0.070
train_acc    mean   0.748
             std    0.031
mean_vs_std         0.104
Name: Mean, dtype: object

In [637]:
filename = "4847-1-1.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.741
             std    0.017
train_acc    mean   0.744
             std    0.012
mean_vs_std         0.023
Name: Mean, dtype: object

In [638]:
filename = "4850-1-2.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.739
             std    0.014
train_acc    mean   0.741
             std    0.009
mean_vs_std         0.020
Name: Mean, dtype: object

In [639]:
filename = "4863-2-0.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.708
             std    0.056
train_acc    mean   0.744
             std    0.028
mean_vs_std         0.083
Name: Mean, dtype: object

In [640]:
filename = "4864-2-1.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.736
             std    0.019
train_acc    mean   0.740
             std    0.010
mean_vs_std         0.027
Name: Mean, dtype: object

In [641]:
filename = "4865-2-2.csv"
df = return_metrics(filename)
df.loc["Mean"]

C:\Users\Master\AppData\Local\Temp\ipykernel_25148\4229097355.py:18: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_metrics.loc["Mean"] = df_metrics.mean()


Name                  NaN
test_acc     mean   0.736
             std    0.014
train_acc    mean   0.738
             std    0.008
mean_vs_std         0.020
Name: Mean, dtype: object

# Metricas CNN

In [642]:
filename = "cnn-5140-0-0.csv"
df = return_metrics_cnn(filename)
df.loc["Mean"]

FileNotFoundError: [Errno 2] No such file or directory: 'cnn-5140-0-0.csv'

In [ ]:
filename = "cnn-5141-0-1.csv"
df = return_metrics_cnn(filename)
df.loc["Mean"]

In [ ]:
filename = "cnn-5145-0-2.csv"
df = return_metrics_cnn(filename)
df.loc["Mean"]

# A futuro

In [ ]:
filename = "future-5057-0-1.csv"
df = return_metrics(filename)
df.loc["Mean"]

In [ ]:
filename = "grid-search.csv"
df = return_metrics_gs(filename)
# df.loc["Mean"]
# df.sort_values(by=[('test_acc', 'mean'), 'mean_vs_std', 'mean_vs_std_loss'], ascending=False)
df